In [ ]:
# import all the PySpark stuff we have to do every time -CLM 20211017 Module 16 challenge deliverable # 2
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Skip this cell if you are going to import the vide_table directly from the LOCAL CSV FILE
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [ ]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [ ]:
from pyspark import SparkFiles
# skip this cell unless you are trying to download the vine table from the CSV file saved in the GitHub repository
# url = "https://github.com/gcmastra/Amazon_Vine_Analysis/blob/ff3b80f839217e9422eac0c8a33c0578910432c5/vine_table1.csv"
# spark.sparkContext.addFile(url)
# vine_df = spark.read.csv(SparkFiles.get("vine_table1.csv"), sep=",", header=True)

In [ ]:
# whichever way you imported the dataframe, now we can answer the four quesions - 
# what is this? Passover?  What makes this night different from any other night?
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [ ]:
# first I had to prove to myself there really were reviews where total votes were greater than 20 and yes there definitely are
# all of these operations on the table take a long time when the query is run against all 2 million rows (row count = 2,302,041)
vine_df.orderBy(vine_df["total_votes"].desc()).show(12)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35XE3G8NNGC9S|          4|         2703|       2751|   N|                N|
| RSQET9EMO7CL3|          1|         2663|       2701|   N|                N|
|R2MHD9H0PWCXE7|          5|         2478|       2600|   N|                Y|
| RKZOCIM2VFFUH|          1|         2449|       2556|   N|                N|
|R18GPM5UHCIQ5C|          5|         2362|       2496|   N|                Y|
| RBGZFXUBOJOIK|          5|         2350|       2420|   N|                Y|
|R22O2PHMT7VDD8|          5|         1864|       1961|   N|                Y|
|R3SMZD3JD2VX2Z|          4|         1802|       1839|   N|                N|
|R2SWUI9VB5TAOK|          5|         1722|       1792|   N|                Y|
|R2DXM6IC77F7UY|          1|         1682|       1735|   N|     

In [ ]:
# df1 row count = 43574
df1 = vine_df.filter("total_votes >= 20")

# the remaining calculations only make sense if there were more than 20 total votes
# df2 - helpful votes greater than half of total votes 
df2 = df1.filter("helpful_votes * 2 > total_votes")

# df3 = rows from df2 where a Vine review was written
df3 = df2.filter(df2["vine"]=="Y")

# df4 = rows where no Vine review was written
df4 = df2.filter(df2["vine"]=="N")




In [ ]:
result_n2 = df4.filter(df["star_rating"]==5).count()


result_p2 = df3.filter(df["star_rating"]==5).count()


result_p1 = df3.count() 
result_n1 = df4.count() 


In [ ]:
# cast as float first 
result_p3 = 52.34
result_n3 = 52.78
# is a fraction of a percent a statistically different result?? 
# print(round(result_p2*100/result_p1,2))
# print(round(result_n2*100/result_n1,2))
df3.show(15)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|
|R1W3KCDHM6BMZT|          4|           37|         38|   Y|                N|
|R3KDMPS96IJ10R|          2|           15|         21|   Y|                N|
|R1XWEI58A7VKLB|          5|          123|        128|   Y|     

In [ ]:
results_df = spark.createDataFrame([("Vine Paid",result_p1,result_p2,"52.34"),("Non Paid",result_n1,result_n2,"52.78")], ["Results","Total Reviews","Num 5 stars","Percent"]).show()
                                                                                 


+---------+-------------+-----------+-------+
|  Results|Total Reviews|Num 5 stars|Percent|
+---------+-------------+-----------+-------+
|Vine Paid|          107|         56|  52.34|
| Non Paid|        39768|      20990|  52.78|
+---------+-------------+-----------+-------+

